# Databases and SQL for Data Science with Python Final Project

This project served as the final project for the course Databases and SQL for Data Science with Python with Honors. Project aims can be found below. 


## Project Set-Up

In [9]:
# import required packages and libraries
import prettytable
import pandas as pd
import sqlite3

prettytable.DEFAULT = 'DEFAULT'

# load the SQL magic module
%load_ext sql

# establish a connection to the database
conn = sqlite3.connect('FinalDB.db')
%sql sqlite:///FinalDB.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


### Load the data and add to SQL database

In [6]:
# load census data
chi_census_data = pd.read_csv('ChicagoCensusData.csv')
print(chi_census_data.head())

# load public school data
chi_pub_school_data = pd.read_csv('ChicagoPublicSchools.csv')
# update column name
chi_pub_school_data = chi_pub_school_data.rename(columns={'Elementary, Middle, or High School': 'SCHOOL_TYPE'})
print(chi_pub_school_data.head())

# load crimes data
chi_crime_data = pd.read_csv('ChicagoCrimeData.csv')
print(chi_crime_data.head())

   COMMUNITY_AREA_NUMBER COMMUNITY_AREA_NAME  PERCENT_OF_HOUSING_CROWDED  \
0                    1.0         Rogers Park                         7.7   
1                    2.0          West Ridge                         7.8   
2                    3.0              Uptown                         3.8   
3                    4.0      Lincoln Square                         3.4   
4                    5.0        North Center                         0.3   

   PERCENT_HOUSEHOLDS_BELOW_POVERTY  PERCENT_AGED_16__UNEMPLOYED  \
0                              23.6                          8.7   
1                              17.2                          8.8   
2                              24.0                          8.9   
3                              10.9                          8.2   
4                               7.5                          5.2   

   PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA  \
0                                          18.2   
1                               

Establish a connection between SQL magic module and the database `FinalDB.db`


In [8]:
# add tables to database
chi_census_data.to_sql('chi_census_data', conn, if_exists='replace', index=False)

chi_pub_school_data.to_sql('chi_pub_school_data', conn, if_exists='replace', index=False)

chi_crime_data.to_sql('chi_crime_data', conn, if_exists='replace', index=False)

# confirm addition of tables to database
%sql SELECT name FROM sqlite_master WHERE type='table';

 * sqlite:///FinalDB.db
Done.


name
chi_census_data
chi_pub_school_data
chi_crime_data


## Project Aims

### Problem 1

##### Find the total number of crimes recorded in the CRIME table.


In [14]:
total_crimes = %sql SELECT COUNT(*) FROM chi_crime_data;

total_crimes_count = total_crimes[0][0]
print('')
print('The total number of crimes is', total_crimes_count)

 * sqlite:///FinalDB.db
Done.

The total number of crimes is 533


### Problem 2

##### List community area names and numbers with per capita income less than 11000.


In [26]:
%sql SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME FROM chi_census_data WHERE PER_CAPITA_INCOME < 11000;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME
26.0,West Garfield Park
30.0,South Lawndale
37.0,Fuller Park
54.0,Riverdale


### Problem 3

##### List all case numbers for crimes involving minors? (children are not considered minors for the purposes of crime analysis) 


In [58]:
%%sql 

SELECT CASE_NUMBER 
FROM chi_crime_data
WHERE DESCRIPTION LIKE '%minor%';

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER
HL266884
HK238408


### Problem 4

##### List all kidnapping crimes involving a child?


In [74]:
%%sql 

SELECT * 
FROM chi_crime_data
WHERE PRIMARY_TYPE = 'KIDNAPPING' AND DESCRIPTION LIKE '%child%';

 * sqlite:///FinalDB.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,0,0,1533,15,29.0,25.0,20,1143050.0,1897546.0,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


### Problem 5

##### List the kind of crimes that were recorded at schools. (No repetitions)


In [66]:
%%sql 

SELECT DISTINCT PRIMARY_TYPE 
FROM chi_crime_data
WHERE LOWER(LOCATION_DESCRIPTION) LIKE '%school%';

 * sqlite:///FinalDB.db
Done.


PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


### Problem 6

##### List the type of schools along with the average safety score for each type.


In [73]:
%%sql

SELECT SCHOOL_TYPE, AVG(SAFETY_SCORE) AS AVG_SAFETY
FROM chi_pub_school_data
GROUP BY SCHOOL_TYPE;

 * sqlite:///FinalDB.db
Done.


SCHOOL_TYPE,AVG_SAFETY
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


### Problem 7

##### List 5 community areas with highest % of households below poverty line


In [35]:
%%sql 

SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY
FROM chi_census_data 
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC
LIMIT 5;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
54.0,Riverdale,56.5
37.0,Fuller Park,51.2
68.0,Englewood,46.6
29.0,North Lawndale,43.1
27.0,East Garfield Park,42.4


### Problem 8

##### Which community area is most crime prone? Display the coumminty area number only.


In [54]:
%%sql 
SELECT COMMUNITY_AREA_NUMBER
FROM chi_crime_data
GROUP BY COMMUNITY_AREA_NUMBER
ORDER BY COUNT(*) DESC
LIMIT 1;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER
25.0


### Problem 9

##### Use a sub-query to find the name of the community area with highest hardship index

<br>(Note from A. Hedden) <br> Simple query would be: 

SELECT COMMUNITY_AREA_NAME <br>
FROM chi_census_data <br>
ORDER BY HARDSHIP_INDEX DESC <br>
LIMIT 1; <br>

In [52]:
%%sql
SELECT COMMUNITY_AREA_NAME
FROM chi_census_data
WHERE COMMUNITY_AREA_NUMBER = (
    SELECT COMMUNITY_AREA_NUMBER
    FROM chi_census_data
    ORDER BY HARDSHIP_INDEX DESC
    LIMIT 1
);

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Riverdale


### Problem 10

##### Use a sub-query to determine the Community Area Name with most number of crimes?


In [75]:
%%sql 
SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME
FROM chi_census_data
WHERE COMMUNITY_AREA_NUMBER = (
    SELECT COMMUNITY_AREA_NUMBER
    FROM chi_crime_data
    GROUP BY COMMUNITY_AREA_NUMBER
    ORDER BY COUNT(*) DESC
    LIMIT 1
);

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME
25.0,Austin


# Final Assignment Author and Contributor Credits
## Author(s)

<h4> Hima Vasudevan </h4>
<h4> Rav Ahuja </h4>
<h4> Ramesh Sannreddy </h4>

## Contribtuor(s)

<h4> Malika Singla </h4>
<h4>Abhishek Gagneja</h4>
<!--
## Change log

| Date       | Version | Changed by        | Change Description                             |
| ---------- | ------- | ----------------- | ---------------------------------------------- |
|2023-10-18  | 2.6     | Abhishek Gagneja  | Modified instruction set |
| 2022-03-04 | 2.5     | Lakshmi Holla     | Changed markdown.                   |
| 2021-05-19 | 2.4     | Lakshmi Holla     | Updated the question                           |
| 2021-04-30 | 2.3     | Malika Singla     | Updated the libraries                          |
| 2021-01-15 | 2.2     | Rav Ahuja         | Removed problem 11 and fixed changelog         |
| 2020-11-25 | 2.1     | Ramesh Sannareddy | Updated the problem statements, and datasets   |
| 2020-09-05 | 2.0     | Malika Singla     | Moved lab to course repo in GitLab             |
| 2018-07-18 | 1.0     | Rav Ahuja         | Several updates including loading instructions |
| 2018-05-04 | 0.1     | Hima Vasudevan    | Created initial version                        |
-->
## <h3 align="center"> © IBM Corporation 2023. All rights reserved. <h3/>
